In [1]:
import numpy as np
from numba import jit

In [2]:
class HMM:
    def forward(A,B,pi,N,M,sequence):
        T=len(sequence)
        alpha=np.zeros([T,N])
        alpha[0]=pi*B[:,sequence[0]]
        t=1
        while True:
            if t==T:
                break
            alpha[t]=alpha[t-1]@A*B[:,sequence[t]]
            t+=1
        return(np.sum(alpha[T-1]))
    def backward(A,B,pi,N,M,sequence):
        T=len(sequence)
        beta=np.zeros([T,N])
        beta[T-1]=1
        t=T-2
        while True:
            if t<0:
                break
            beta[t]=A@(B[:,sequence[t+1]]*beta[t+1])
            t-=1
        return(np.sum(pi*B[:,sequence[0]]*beta[0]))
    def Viterbi(A,B,pi,N,M,sequence):
        T=len(sequence)
        delta=np.zeros([T,N])
        psi=np.zeros([T,N])
        delta[0]=pi*B[:,sequence[0]]
        t=1
        while True:
            if t==T:
                break
            delta_A=delta[t-1,np.newaxis].T*A
            delta[t]=np.max(delta_A,axis=0)*B[:,sequence[t]]
            psi[t]=np.argmax(delta_A,axis=0)
            t+=1
        psi=psi.astype(int)
        q=np.zeros(T).astype(int)
        q[T-1]=np.argmax(delta[T-1])
        t=T-2
        while True:
            if t<0:
                break
            q[t]=psi[t+1,q[t+1]]
            t-=1
        return(q)

In [3]:
M=3
N=3
pi=np.array([.3,.3,.4])
A=np.array([[.2,.3,.5],[.1,.5,.4],[.6,.1,.3]])
B=np.array([[0.1,0.5,0.4],[0.2,0.4,0.4],[0.3,0.6,0.1]])
sequence=[0,1,2]
print(HMM.forward(A,B,pi,N,M,sequence))
print(HMM.forward(A,B,pi,N,M,sequence))
print(HMM.Viterbi(A,B,pi,N,M,sequence))

0.030162
0.030162
[2 2 0]
